# The Price is Right

We'll build a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [01:49<00:00, 21.95s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

'Title: Walmart Flash Deals: Up to 65% off + free shipping w/ $35\nDetails: The Walmart Flash Deals section has refreshed today with price drops on thousands of items. Shop deals by category or scroll through the featured section to find the best flash deals this week. Shop Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Walmart-Flash-Deals-Up-to-65-off-free-shipping-w-35/21733479.html?iref=rss-c196'

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price.

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price.

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price.

Deals:

Title: LG TVs and Soundbar Daily Deals at Best Buy: Up to $1,000 off + free shipping
Details: Today only, Best Buy discounts a selection of LG TVs and home theater systems. TV prices start at $299.99 ($400 off) for the LG 55" 55NANO75UQA Class NanoCell 75UQA Series LED 4K UHD Smart TV. Shop Now at Best Buy
Features: 
URL: https://www.dealnews.com/LG-TVs-and-Soundbar-Daily-D

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The Turnstile Audio Concourse Series Multi-Pattern Condenser Microphone is designed for professional sound capture, featuring low self-noise performance that allows for clean audio recordings. It includes a built-in 10dB pad and low-cut switch, making it versatile across different recording environments. This microphone can operate in cardioid, omnidirectional, and figure-8 patterns, accommodating various recording techniques and scenarios. The model TAC700 is an excellent choice for aspiring musicians and content creators.', price=40.0, url='https://www.dealnews.com/products/Turnstile/Turnstile-Audio-Concourse-Series-Multi-Pattern-Condenser-Microphone/487258.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The Yamaha True Wireless Earbuds provide an immersive listening experience with Bluetooth 5.2 technology, ensuring a stable and efficient connection. They come with a compact charging case, featuring five sets of eartips for a customizable fit, allowing for comfort during long use. With an impressive sound quality and a sleek design, these earbuds are ideal for music lovers and professionals alike.', price=65.0, url='https://www.dealnews.com/products/Yamaha/Yamaha-True-Wireless-Earbuds/459553.html?iref=rss-c142'), Deal(product_description='The Turnstile Audio Concourse Series Multi-Pattern Condenser Microphone is designed for versatility and high-quality sound capture. It features three selectable polar patterns: cardioid, omnidirectional, and figure-8, allowing for various recording situations. Additionally, the microphone has a built-in 10dB pad and low-cut switch, making it suitable for both studio and live environments, effectively red